# Tutorial (Part 1): Data Handling Module
This tutorial details the conversion of spatial omics data into a format compatible with SORBET, using the `data_handling` module. The tutorial uses a subset of data from the CosMx Metastatic Melanoma Dataset. It is stored in `data/tutorial/raw_data/`.

This tutorial is split intro three steps:
1. Load the data into an `OmicsGraph` (`SORBET/data_handling/graph_model.py:9`) object. This is the core class useful for processing and analyzing data. For each tissue sample, the data handling code data requires:
    - A list of cells
    - A list of cell-cell edges
    - An array of cell profiles
    - A list of markers (*e.g.,* genes)
    - A graph label
    - (Optionally) Additional metadata values
2. Normalize input graphs using different methods (*e.g.,* log normalize or z-normalization)
3. Extract subgraphs from each graph.

**n.b.** Subsequent tutorials handle learning appropriate models (`Tutorials_2_Learning.ipynb`) and reasoning over those modules (`Tutorial_3_Reasoning.ipynb`). This module should be run prior to starting those modules.

In [5]:
import os
import csv, pickle
from tqdm.notebook import tqdm as tqdm
from functools import partial
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.manifold import TSNE, MDS
from sklearn.decomposition import PCA

import numpy as np
import pandas as pd

import torch
import torch_geometric
import ray

In [2]:
%load_ext autoreload
%autoreload 2

In [2]:
import SORBET
import SORBET.data_handling as data_handling

In [ ]:
input_data_dirpath = "data/tutorial/input_data/" # Subset of Melanoma dataset included with GitHub Repo.

## Convert Data to SORBET's Format:

First, data is converted from any pre-existing format to a graph-structured format used in SORBET. This is a class, `data_handling.OmicsGraph`, included in `SORBET/data_handling/graph_model`. 

The function's docstring is:
```python
class OmicsGraph:
    """
    Base class encoding a spatial graph representation of a profiled sample. 
    
    Facilitates two major graph operations: get_khop_neighborhood and make_subgraph. 
        get_khop_neighborhood: a function to extract the k-hop neighborhood around a chosen node
        make_subgraph: makes a subgraph out of a specified list of nodes

    Attributes:
        graph: a networkx graph encoding the spatial relationships of cells
        node_attributes: a dictionary mapping nodes to marker values
        markers: markers profiled in the sample
        vertices: list of vertices (or nodes) in the sample.
        graph_label: integer encoding the attributed label of the graph
        meta_markers: list of markers associated with the cell not used in core inference.
            An example would be proteins measured simultaneously with transcriptomics data.
        meta_markers_data: dictionary mapping vertices to meta markers (shared order with meta_markers) 
        node_meta_attributes: parallels the node_attributes structure for the meta_marker subset
    """

    def __init__(self, vertex_lst: list, edge_lst: list, data: np.ndarray, markers: list, graph_label: int, meta_markers: Union[list, dict] = None):
        """Intializes the OmicsGraph object.

        Args:
            vertex_lst: list of vertices included in the sample
            edge_lst: list of vertex tuples encoding (graph) relationships between vertices 
            data: NumPy array encoding (vertex_lst) x (markers) data 
            markers: list of markers measured in the sample
            graph_label: integer encoding the relevant value associated with the
            meta_markers: list of meta-markers and dictionary of vertex association, as described in the class definition 
        """
```
(See `SORBET/data_handling/graph_model.py:9-39`)

In [73]:
output_data_dirpath= "data/tutorial/graphs_py/complete/"
if not os.path.exists(output_data_dirpath):
    os.makedirs(output_data_dirpath)

In [74]:
# The CosMx data includes both transcriptomic and proteomic measurements. The proteomics data is treated as metadata
# It includes the S100B marker, which we will use to identify subgraphs.
cell_profile_data = pd.read_csv(os.path.join(input_data_dirpath, "cell_profiles.csv"))
cell_profile_metadata = pd.read_csv(os.path.join(input_data_dirpath, "cell_metadata_profiles.csv")) 

transcriptomic_marker_list = np.array(cell_profile_data.columns[2:])
print("Transcriptomics Markers (first 5)", transcriptomic_marker_list[:5])
proteomic_markers_list = np.array(cell_profile_metadata.columns[2:])
print("Proteomics Markers (first 5)", proteomic_markers_list[:5])

graph_edges_data = pd.read_csv(os.path.join(input_data_dirpath, "graph_edges.csv"))
graph_labels_data = pd.read_csv(os.path.join(input_data_dirpath, "graph_labels.csv"))

graph_paths_list = list()
for graph_id, graph_label in graph_labels_data.values:
    print(f'Converting graph {graph_id} to an OmicsGraph')
    # Extract cell data and metadata for the graph:
    _cell_data = cell_profile_data[cell_profile_data["Graph_ID"] == graph_id][transcriptomic_marker_list].to_numpy()
    _cell_metadata = cell_profile_metadata[cell_profile_data["Graph_ID"] == graph_id][proteomic_markers_list].to_numpy()
    _cell_list = cell_profile_data[cell_profile_data["Graph_ID"] == graph_id]["Cell_ID"].tolist()

    # Check that the metadata and data share the same order:
    _data_order = cell_profile_data[cell_profile_data["Graph_ID"] == graph_id]["Cell_ID"]
    _metadata_order = cell_profile_metadata[cell_profile_metadata["Graph_ID"] == graph_id]["Cell_ID"]
    assert np.all(_data_order == _metadata_order)
    
    # Extract the edges in the graph:
    _graph_edges = graph_edges_data[graph_edges_data["Graph_ID"] == graph_id][["Cell_ID_1", "Cell_ID_2"]].to_numpy().tolist()

    # Create an OmicsGraph Object:
    omics_graph = data_handling.OmicsGraph(
        _cell_list,                                 # List of cell ID's.
        _graph_edges,                               # List of edges between vertices (corresponding with indexing in _cell_list).
        _cell_data,                                 # Profiles for each cell.
        transcriptomic_marker_list,                 # List of marker names.
        graph_label,                                # Binary label. Integer in {0,1}
        (proteomic_markers_list, _cell_metadata)    # Metadata. Passed as tuple of marker names and marker values
    )

    # Dump complete graph to a chosen output location.
    output_fpath = os.path.join(output_data_dirpath, f'{graph_id}.p')
    data_handling.dump_omicsgraph(omics_graph, output_fpath)
    
    
    label_meaning = "Responder" if graph_label == 1 else "Non-responder" # Dataset specific; not necessary
    graph_paths_list.append((graph_id, graph_label, label_meaning, output_fpath))    

labels_fpath = "data/tutorial/labels.csv"
with open(labels_fpath, 'w+') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(["Graph ID", "Label", "Label Meaning", "Complete Graph"])
    writer.writerows(graph_paths_list)

Transcriptomics Markers (first 5) ['AATK' 'ABL1' 'ABL2' 'ACE' 'ACE2']
Proteomics Markers (first 5) ['Mean.MembraneStain' 'Max.MembraneStain' 'Mean.S100B' 'Max.S100B'
 'Mean.CD45']
Converting graph run_5612_fov_1 to an OmicsGraph
Converting graph run_5612_fov_2 to an OmicsGraph
Converting graph run_5612_fov_3 to an OmicsGraph
Converting graph run_5612_fov_4 to an OmicsGraph
Converting graph run_5612_fov_5 to an OmicsGraph
Converting graph run_5612_fov_6 to an OmicsGraph
Converting graph run_5612_fov_7 to an OmicsGraph
Converting graph run_5612_fov_8 to an OmicsGraph
Converting graph run_5612_fov_9 to an OmicsGraph
Converting graph run_5612_fov_11 to an OmicsGraph


## Normalize Datasets:

Next, we normalize the input data. There are multiple different options for modifying the data. Here, we will illustrate one of the normalization calls.

The methods for normalizing `OmicsGraph` objects are included in `SORBET/data_handling/normalize.py`. The two methods either normalize graphs individually (`data_handling.normalize_graph`) or across the entire dataset (`data_handling.normalize_dataset`; *e.g.,* PCA on all graphs). *Note*, there are multiple potential options for normalizing datasets.

**n.b.** Graphs are modified in place.

```Python
def normalize_graph(tissue_graph: OmicsGraph, normalize_method: str, normalize_args: dict = dict()) -> None:
    """Normalize a single sample. OmicsGraph objects are modified in place. 

    Shallow wrapper around _normalize_[x] methods. Multiple normalization methods included including: 
        'total_count': _normalize_by_total_count
        'log_normalize': _normalize_by_log,
        'pca': _normalize_by_pca,
        'z-normalize': _normalize_by_variance,
        'to-range': _normalize_to_range

    Additional args may be passed in the normalize_args dictionary. See specific functions for options.
    
    Args:
        tissue_graph: an OmicsGraph object to normalize
        normalize_method: the chosen method to normalize a graph
        normalize_args: dictionary including additional arguments, specific to each method.

    """
```
(See `SORBET/data_handling/normalize.py:13-30`)

In [75]:
output_data_dirpath= "data/tutorial/graphs_py/complete/"
lognormalized_data_dirpath = "data/tutorial/graphs_py/lognormalized/"
znormalized_data_dirpath = "data/tutorial/graphs_py/znormalized/"

for dirpath in [lognormalized_data_dirpath, znormalized_data_dirpath]:
    if not os.path.exists(dirpath): os.makedirs(dirpath)

In [76]:
log_graphs, fnames = list(), list()
for ifile in os.listdir(output_data_dirpath):
    igraph_fpath = os.path.join(output_data_dirpath, ifile)

    # Load OmicsGraph object
    omics_graph = data_handling.load_omicsgraph(igraph_fpath)
    
    # Graphs are modified in-place:
    data_handling.normalize_graph(omics_graph, "log_normalize") # Normalized in-place
    data_handling.dump_omicsgraph(omics_graph, os.path.join(lognormalized_data_dirpath, ifile))

    # Append to the log graphs
    log_graphs.append(omics_graph)
    fnames.append(ifile)

data_handling.normalize_dataset(log_graphs, "z-normalize") # List normalized in-place
for fname, graph in zip(fnames, log_graphs):
    data_handling.dump_omicsgraph(graph, os.path.join(znormalized_data_dirpath, fname))

## Subgraph Extraction Algorithm:

Next, we extract subgraphs from each of the graphs. This method should typically be accessed using `data_handling.create_subgraphs` in `data_handling/preprocess.py:43`. This code wraps `data_handling.subgraph_extraction` in `data_handling/subgraph_extraction.py`. 

```Python
def subgraph_extraction(tissue_graph: OmicsGraph, extraction_method: str, extraction_args: dict = dict()) -> List[OmicsGraph]:
    """Implements a basic interface to access different subgraph extraction methods (below)
    
    ...
    
    Args:
        tissue_graph: OmicsGraph object, which the subgraph extraction algorithm will be applied on
        extraction_method: chosen method. Options defined above.
        extraction_args: dictionary with additional arguments passed to the subgraph extraction algorithm
    
    Returns:
        A list of OmicsGraph objects representing the extracted subgraphs. 
    """
```
(See `SORBET/data_handling/subgraph_extraction.py:8-25`)

The final step is to convert the OmicsGraph objects into a torch subgraphs for ease of use with learning.

In [77]:
K = 10 # Neighborhood size
MS = 50 # Minimum size
anchor_marker =  "Mean.S100B"

input_data_dir = "data/tutorial/graphs_py/znormalized/"
output_data_dir = "data/tutorial/processed_subgraphs/znormalized/"
if not os.path.exists(output_data_dir): os.makedirs(output_data_dir)

# Can change complete_graphs_dirpath to different normalization techniques
data_handling.create_subgraphs(input_data_dir, output_data_dir,                     # Input and output directories.
                               "microenvironment",                                  # Extraction algorithm. Multiple options. 
                               {"marker": anchor_marker, "k": K, "minimum_size":MS} # Additional arguments passed to the extraction algorithm
                              )

/usr/local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
# Convert the OmicsGraph data objects to torch subgraphs
data_handling.create_torch_subgraphs(output_data_dir) 